In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"

execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb
reload(pu_delayfb)
import analysis.lib.fastcarboncontrol.fcc as fcc
reload(fcc)
import matplotlib as mpl
mpl.style.use('seaborn-bright')
%matplotlib inline

In [ ]:
msmts = {
    "Q-DPS": {
        "ots": [
            "20170720155000",
            "20170720174400",
        ],
        "tomos": [
            "XX",
            "YY",
            "ZZ",
            "XY",
            "YX",
        ]
    },
    "Q-DES": {
        "ots": [
            "20170720163100",
        ],
        "tomos": [
            "XX",
            "YY",
            "ZZ",
            "XY",
            "YX",
        ]
    },
    "cl-1" : {
        "ots": [
            "20170720170600",
        ],
        "tomos": [
            "XI",
            "IX",
            "XX"
        ],
    },
    "cl-2": {
        "ots": [
            "20170720172200"
        ],
        "tomos": [
            "45.045.0",
            "45.0-45.0"
        ]
    }
}

In [ ]:
import copy
msmts_and_data = copy.copy(msmts)

for m in msmts.iterkeys():
    print("Msmt: "+m)
    ots = msmts[m]["ots"]
    msmts_and_data[m]["tomos_data"] = dict()
    for t in msmts[m]["tomos"]:
        print("Tomo: %s - %s" % (m, t))
        data = dict()
        data["x"], data["y"], data["y_u"], data["fr"] = \
        pu_delayfb.number_of_repetitions_stitched(
            contains="phase_fb_delayline",
            older_thans=ots,
            do_fit=True,
            x_only=True,
            tomo_basis=t,
            fit_n=1.0,
            fixed=[0,2,5,6,4],
            ret_data_fit=True
        )
        msmts_and_data[m]["tomos_data"][t] = data

# Quantum DPS measurements

In [ ]:
fig = plt.figure()
ax = plt.subplot()

tomos_data = msmts_and_data["Q-DPS"]["tomos_data"]

i = 0

tomo_order = ["XX", "YY", "ZZ", "XY", "YX"]
fit_xvals = np.linspace(0.,1000., 2000.)

for t in tomo_order:
    plt.errorbar(
        tomos_data[t]["x"], tomos_data[t]["y"], tomos_data[t]["y_u"], 
        fmt='o', color="C%d" % i, 
        label = r"$\langle %s \rangle$, $N_{1/e} = %d \pm %d$" % (t, np.round(tomos_data[t]["fr"]['params_dict']['T']),
                                                                 np.round(tomos_data[t]["fr"]['error_dict']['T']))
    )
    
    plt.plot(fit_xvals, tomos_data[t]["fr"]['fitfunc'](fit_xvals), color="C%d" % i)
    
    i += 1
    
plt.ylim(-1,1)
plt.xlim(0,1000)

plt.xlabel("Number of LDE attempts")
plt.ylabel(r"Expectation value")

lgd = plt.legend(loc=2, bbox_to_anchor=(1,1))

plt.savefig("quantum-dps-storage.png", dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')

# Quantum DES measurements

In [ ]:
fig = plt.figure()
ax = plt.subplot()

tomos_data = msmts_and_data["Q-DES"]["tomos_data"]

i = 0

tomo_order = ["XX", "YY", "ZZ", "XY", "YX"]
fit_xvals = np.linspace(0.,1000., 2000.)

for t in tomo_order:
    plt.errorbar(
        tomos_data[t]["x"], tomos_data[t]["y"], tomos_data[t]["y_u"], 
        fmt='o', color="C%d" % i, 
        label = r"$\langle %s \rangle$, $N_{1/e} = %d \pm %d$" % (t, np.round(tomos_data[t]["fr"]['params_dict']['T']),
                                                                 np.round(tomos_data[t]["fr"]['error_dict']['T']))
    )
    
    plt.plot(fit_xvals, tomos_data[t]["fr"]['fitfunc'](fit_xvals), color="C%d" % i)
    
    i += 1
    
plt.ylim(-1,1)
plt.xlim(0,1000)

plt.xlabel("Number of LDE attempts")
plt.ylabel(r"Expectation value")

lgd = plt.legend(loc=2, bbox_to_anchor=(1,1))

plt.savefig("quantum-des-storage.png", dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')

# Classical measurements

In [ ]:
fig = plt.figure()
ax = plt.subplot()

tomos_data = msmts_and_data["cl-1"]["tomos_data"].copy()
tomos_data.update(msmts_and_data["cl-2"]["tomos_data"])

i = 0

tomo_order = ["IX","XI","XX","45.0-45.0","45.045.0"]
labels = ["IX", "XI", "XX", "X+45\degree, X-45\degree", "X+45\degree, X+45\degree"]
fit_xvals = np.linspace(0.,1000., 2000.)

for t in tomo_order:
    plt.errorbar(
        tomos_data[t]["x"], tomos_data[t]["y"], tomos_data[t]["y_u"], 
        fmt='o', color="C%d" % i, 
        label = r"$\langle %s \rangle$, $N_{1/e} = %d \pm %d$" % (labels[i], np.round(tomos_data[t]["fr"]['params_dict']['T']),
                                                                 np.round(tomos_data[t]["fr"]['error_dict']['T']))
    )
    
    plt.plot(fit_xvals, tomos_data[t]["fr"]['fitfunc'](fit_xvals), color="C%d" % i)
    
    i += 1
    
plt.ylim(-1,1)
plt.xlim(0,1000)

plt.xlabel("Number of LDE attempts")
plt.ylabel(r"Expectation value")

lgd = plt.legend(loc=2, bbox_to_anchor=(1,1))

plt.savefig("classical-des-dps-storage.png", dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')